In [ ]:
%%capture
!pip install -q streamlit accelerate transformers ctransformers[cuda]
!npm install -g localtunnel

## Prosta aplikacja z wykorzystaniem Streamlit i Bielika w wersji skwantyzowanej GGUF (Q4)
Metoda kwantyzacji: Group-wise Gradient Update Filtering (GGUF)
Rozmiar: Q4_K_M


In [ ]:
%%writefile app.py
import streamlit as st
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, TextIteratorStreamer
from threading import Thread

@st.cache_resource      # Dekorator, żeby model i inne rzeczy były trzymane w pamięci cache
def prepare_model() -> tuple:
    tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-7B-Instruct-v0.1")

    model = AutoModelForCausalLM.from_pretrained(
        "speakleash/Bielik-7B-Instruct-v0.1-GGUF",
        model_file="bielik-7b-instruct-v0.1.Q4_K_M.gguf",
        model_type="mistral", gpu_layers=50, hf=True)

    streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    return tokenizer, model, streamer


def generate_text(tokenizer, model, streamer, prompt, max_tokens = 512) -> str:
    # Przygotowanie promptu przez tokenizer
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Przygotowanie zmiennych dla modelu
    generation_kwargs = {
        "input_ids": inputs["input_ids"],
        "max_new_tokens": max_tokens,
        "streamer": streamer
    }

    # Uruchamiamy generowanie modelu w osobnym wątku
    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    for output in streamer:
        yield output    # Zwracamy fragmenty odpowiedzi


### ------------------------------------------------------------------------ ###
# Aplikacja Streamlit

# Historia czatu
if "messages" not in st.session_state:
    st.session_state.messages = []


st.title("Bielik-v0.1-7B (Q4) LLM App")

# Ładowanie modelu
with st.status("Pobieranie i przygotowanie modelu...", expanded=True) as status:
    tokenizer, model, streamer = prepare_model()
    status.update(label="Model gotowy do rozmowy!", state="complete", expanded=False)

# Historia konwersacji
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Czat z modelem
if human_prompt := st.chat_input("Witaj, o czym dzisiaj porozmawiamy?"):
    # Dodanie specjalnych tokenów, żeby model działał poprawnie
    prompt = f"<s>[INST]{human_prompt}[/INST]"

    # Zapisanie promptu do historii
    st.session_state.messages.append({"role": "user", "content": human_prompt})

    # Pokazanie wiadomości od użytkownika
    with st.chat_message("user"):
        st.markdown(human_prompt)

    # Odpowiedź modelu
    with st.chat_message("assistant"):
        # Streaming - czyli wiadomość będzie uzupełniana jak tylko model zwróci kolejny token
        # czyli będą się pojawiać kolejne tokeny i nie będziemy czekać na całą wiadomość
        response_stream = st.write_stream(generate_text(tokenizer, model, streamer, prompt))

    # Zapisanie odpowiedzi modelu do historii czatu
    st.session_state.messages.append({"role": "assistant", "content": response_stream})

In [ ]:
# Po uruchomieniu będzie adres IP - należy go skopiować i przejść na stronę, która będzie poniżej
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com